In [2]:
import sys, os
sys.path.append('..')

# Verify the path is correct
print(f"Current directory: {os.getcwd()}")
print(f"Parent directory exists: {os.path.exists('..')}")
print(f"Utils directory exists: {os.path.exists('../utils')}")

from utils.prompts import render
from utils.llm_client import LLMClient #pick llm client
from utils.logging_utils import log_llm_call #log every api call
from utils.router import  pick_model, should_use_reasoning_model #general, reasoning, strrong



Current directory: c:\Users\viraj\Zuu\AI_Engineer_Bootcamp\notebooks
Parent directory exists: True
Utils directory exists: True


In [2]:
os.getcwd()

'c:\\Users\\viraj\\Zuu\\AI_Engineer_Bootcamp\\notebooks'

### 01- Zero Shot

In [21]:
prompt_text , spec = render(
    "zero_shot.v1",
    role = "sentiment_analyst",
    instruction = "Analyze the following text and determine the sentiment as positive/negative/neutral",
    constraints = "the sentiment should be one of the following: positive, negative, neutral",
    format = "the sentiment is : {sentiment}"
    )


model = pick_model("groq", "general")
llm = LLMClient("groq", model)

text = "It seems nice but i dont like it "
messages=[
        {"role": "system", "content": "You must respond in EXACTLY this format with no additional text: 'the sentiment is : [positive/negative/neutral]'"},
        {"role": "user", 
        "content": f"{prompt_text} Review: {text}"
        }
    ]

llm.chat(messages, temperature=0.0)

{'text': 'the sentiment is : neutral',
 'usage': {'input_tokens_est': 101,
  'context_tokens_est': 0,
  'total_est': 104,
  'prompt_tokens_actual': 130,
  'completion_tokens_actual': 6,
  'total_tokens_actual': 136},
 'latency_ms': 420,
 'raw': ChatCompletion(id='chatcmpl-179264fe-847e-4119-9c71-939bc80143a9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='the sentiment is : neutral', role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1769920195, model='llama-3.1-8b-instant', object='chat.completion', mcp_list_tools=None, service_tier='on_demand', system_fingerprint='fp_4387d3edbb', usage=CompletionUsage(completion_tokens=6, prompt_tokens=130, total_tokens=136, completion_time=0.007807293, completion_tokens_details=None, prompt_time=0.007174466, prompt_tokens_details=None, queue_time=0.050534224, total_time=0.014981759), usage_breakdown=None, x_groq=XGroq(id='req_

### 02_Few Shot

provide an exmple

In [ ]:
examples = """
example 01:
review: "I absolutely love this product! It has changed my life for the better."
sentiment: positive

example 02:
review: "This is the worst purchase I've ever made. Completely disappointed."
sentiment: negative

example 03:
review: "The product is okay, nothing special but not terrible either."
sentiment: neutral

example 04:
review: "im really happy with the product its bad"  
sentiment: negative

example 05:
review: "The service was excellent and the staff were very friendly."
sentiment: positive

example 06:
review: "im really unhappy with the product but its amazing"  
sentiment: positive
"""

prompt_text , spec = render(
    "few_shot.v1",
    role = "sentiment_analyst",
    examples = examples,
    query = "",
    constraints = "the sentiment should be one of the following: positive, negative, neutral",
    format = "the sentiment is : {sentiment}"
    )


model = pick_model("groq", "general")
llm = LLMClient("groq", model)

text = "It seems nice but i dont like it"
messages=[
        {"role": "system", "content": "You must respond in EXACTLY this format with no additional text: 'the sentiment is : [positive/negative/neutral]'"},
        {"role": "user", 
        "content": f"{prompt_text} Review: {text}"
        }
    ]

llm.chat(messages, temperature=0.0)

{'text': 'the sentiment is : negative',
 'usage': {'input_tokens_est': 236,
  'context_tokens_est': 0,
  'total_est': 239,
  'prompt_tokens_actual': 265,
  'completion_tokens_actual': 6,
  'total_tokens_actual': 271},
 'latency_ms': 505,
 'raw': ChatCompletion(id='chatcmpl-ab6f42e8-d4ca-4517-8853-411c8ca84e9f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='the sentiment is : negative', role='assistant', annotations=None, executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1769920183, model='llama-3.1-8b-instant', object='chat.completion', mcp_list_tools=None, service_tier='on_demand', system_fingerprint='fp_1151d4f23c', usage=CompletionUsage(completion_tokens=6, prompt_tokens=265, total_tokens=271, completion_time=0.008887368, completion_tokens_details=None, prompt_time=0.016455775, prompt_tokens_details=None, queue_time=0.054586458, total_time=0.025343143), usage_breakdown=None, x_groq=XGroq(id='re

### 03_COT

In [4]:
# CoT auto-routes to reasoning model
reasoning_model = pick_model('groq', 'cot')
print(f'Using reasoning model: {reasoning_model}')

client_reasoning = LLMClient('groq', reasoning_model)

# Problem from live class: break time vs travel time confusion
problem = """A car travels 100 miles in 2 hours. 

Question 1: What is the average speed of the car?
Question 2: If the car stopped for 40 minutes during this 2-hour journey, 
what was the average speed during the actual driving time?

Important: The 2 hours already includes the 40-minute stop."""

# Additional guidance
instruction = """Solve the following problem step by step.
1. First identify whether the car travelled the entire time without stopping or not.
2. If car stopped for x minutes and overall travelled for y hours, the actual driving duration is y-x.
3. If stopping time x is mentioned, do not add it to the travel duration because it's already included.
   So actual travel time is y (total time) - x (stopping time).
4. If car travelled the entire time without stopping, then average speed is distance / y.
5. If car stopped for x minutes, then average speed during driving is distance / (y - x)."""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='math tutor',
    problem=problem
)

# Combine problem with instruction 
full_prompt = f"""text: {prompt_text}

instruction: {instruction}"""

messages = [{'role': 'user', 'content': full_prompt}]
response = client_reasoning.chat(messages, temperature=spec.temperature, max_tokens=spec.max_tokens)

print('CoT Response (Travel Time with Break):')
print('=' * 80)
print(response['text'])
print('=' * 80)
print(f"\nUsage: {response['usage']}")
print(f"Latency: {response['latency_ms']}ms")
log_llm_call('groq', reasoning_model, 'cot', response['latency_ms'], response['usage'])

Using reasoning model: llama-3.3-70b-versatile
CoT Response (Travel Time with Break):
## Step 1: Identify if the car stopped during the journey and the total travel time.
The car stopped for 40 minutes during the 2-hour journey.

## Step 2: Convert the stopping time into hours to calculate the actual driving time.
40 minutes is equal to 40/60 = 2/3 hours.

## Step 3: Calculate the actual driving time by subtracting the stopping time from the total travel time.
Actual driving time = 2 hours - 2/3 hours = 2 - 2/3 = (6/3) - (2/3) = 4/3 hours.

## Step 4: Calculate the average speed for the entire journey.
Average speed for the entire journey = Total distance / Total time = 100 miles / 2 hours = 50 miles per hour.

## Step 5: Calculate the average speed during the actual driving time.
Average speed during driving = Total distance / Actual driving time = 100 miles / (4/3 hours) = 100 * (3/4) = 75 miles per hour.

## Step 6: Answer the questions based on the calculations.
Question 1: The ave